# PNNS Groups (Labels to predict)
Last modification : 13/05/2021

Short description :

    - Clean pnns groups 1 and 2.
    - Search unknown pnns 1 and 2.
    - Find a pnns 3 and 4 when possible.
    - Export csv as 'df_multi_pnns_v2.csv'

## Imports

### Libraries

In [2]:
from eml.datanavig import off_columns_dict
cols = off_columns_dict.copy()
from eml.datanavig import *
from eml.functions import *

In [3]:
import pandas as pd
import numpy as np 
import re
import time 
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

#robotoff
from robotoff.products import ProductDataset
from robotoff.taxonomy import get_taxonomy

from IPython.lib.deepreload import reload as dreload
from pandas import json_normalize

#Settings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
sns.set()
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Full OFF database

In [5]:
%%time
df_raw = pd.read_csv(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\data\en.openfoodfacts.org.products.csv', sep='\t', low_memory=False)
df_raw.shape

Wall time: 1min 20s


(1625068, 184)

In [6]:
df = df_raw.dropna(subset=['categories_tags'])
df.shape

(808019, 184)

### Taxonomy json reference

In [7]:
df_tax = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\data\taxonomy_pnns.json')
df_tax.head()

,all_taxonomy_possibilities,pnns,taxonomy_suggestion
0,{'en:appetizers': 0},appetizers,en:appetizers
1,"{'en:artificially-sweetened-beverages': 2, 'en...",artificially sweetened beverages,en:artificially-sweetened-beverages
2,"{'en:biscuits': 10, 'en:biscuits-and-cakes': 2...",biscuits and cakes,en:biscuits-and-cakes
3,"{'en:bagel-breads': 7, 'en:baguette-made-with-...",bread,en:breads
4,"{'en:breakfast-cereals': 1, 'en:breakfast-cere...",breakfast cereals,en:breakfast-cereals


## Navigation into labels (pnns groups)

### PNNS description

Pnns are the "high level" categories in the taxonomy and the list of labels to predict.
There are 2 main groups of pnns :

    - Group_1 : 9 possible labels (example : beverages, composite food, sugary snacks...)
    - Group_2 : 38 more precises labels (example : fruits nectar, biscuits and cakes, breakfast cereals...)
There is one unique group_1 for every product, but a product can have more than one  group_2.
In other notebooks/documents, sometimes we denote groups 1 and 2 as $G_1$ and $G_2$.

### Labels strings cleaning

In [8]:
df['pnns_groups_2'] = df['pnns_groups_2'].str.lower()
df['pnns_groups_1'] = df['pnns_groups_1'].str.lower()
df['pnns_groups_1'] = df['pnns_groups_1'].str.replace('-',' ')
df.pnns_groups_1.value_counts()

unknown                    154910
sugary snacks              140653
milk and dairy products     82904
fish meat eggs              82817
cereals and potatoes        74820
fat and sauces              63479
beverages                   61758
fruits and vegetables       50780
composite foods             50224
salty snacks                30263
Name: pnns_groups_1, dtype: int64

### Function find_pnns

In [9]:
def find_pnns(row=None, df_taxonomy=None, pnns_n=1, nb_groups=4, search_duplicates=True):
    """
    Find a pnns group based on categories tags and taxonomy suggestions.

    Parameters:
    -----------
        - row: df row to apply
        - df_taxonomy: pd.DataFrame with pnns suggestions & matching tags
        - pnns_n:
        - nb_groups:
        - search_duplicates: 

    Return:
    -------

    Example:
    --------

    """
    #---- Setup variables ----

    #setup idx desired
    pnns_index = pnns_n - 1 
    #setup list of pnns candidates
    pnns_candidates = [] 
    #setup output
    output = np.nan 

    #---- Search Categories Tags ----

    #convert row to a list of tags
    tags_list = row['categories_tags'].split(',') 
    #search parents in taxonomy
    tax_list = [get_taxonomy_info(tags_list[i], info_type='parents') for i in range(len(tags_list))]
    #keep only tags
    tax_list = [item for item in tax_list if not isinstance(item, float)] 
    #convert list of lists to list
    tax_list = [item for sublist in tax_list for item in sublist] 
    #add original tags
    tax_list = tax_list + tags_list 
    #clean strings
    tax_list = [tax.strip(' ').strip('') for tax in tax_list]
    #remove duplicates
    tax_list = list(set(tax_list)) 

    #---- Search pnns candidates ----

    #find candidates in main suggestions
    for suggestion, pnns in zip(df_taxonomy.taxonomy_suggestion, df_taxonomy.pnns): 
        if suggestion in tax_list: pnns_candidates.append(pnns)
    #find candidates in others possibilities
    for possibilities, pnns in zip(df_taxonomy.all_taxonomy_possibilities, df_taxonomy.pnns): 
        for possibility in possibilities.keys():
            if possibility in tax_list: pnns_candidates.append(pnns)

    #---- Setup pnns output ----

    #return nan if no pnns founded
    if not len(pnns_candidates): return output
    #return unique pnns if only one founded
    elif len(pnns_candidates) == 1: 
        output = pnns_candidates[0]
    #return pnns 1 or more
    elif len(pnns_candidates) > 1: 
        output = pnns_candidates[pnns_index]
    
    #---- Search duplicates option ----

    #If pnns already exist in another group
    if search_duplicates:
        existing_values = [row[f'pnns_groups_{i}'] for i in range (1, nb_groups)]
        if output in existing_values: 
        #try to find another pnns
            for i in range(len(pnns_candidates)):
                output = pnns_candidates[i]
                if output not in existing_values:
                    break
    #return output
    return output

### Apply function

In [10]:
%time df['pnns_groups_2'].loc[df['pnns_groups_2']== 'unknown'] = df.apply(lambda row: find_pnns(row=row, df_taxonomy=df_tax, pnns_n=1, nb_groups=2, search_duplicates=False), axis=1)

Wall time: 6min 14s


In [11]:
df.loc[df['pnns_groups_2'] == 'pizza pies and quiches']['pnns_groups_2'] = 'pizza pie and quiches'

In [12]:
%time df['pnns_groups_3'] = df.apply(lambda row: find_pnns(row=row, df_taxonomy=df_tax, pnns_n=2, nb_groups=2), axis=1)
%time df['pnns_groups_4'] = df.apply(lambda row: find_pnns(row=row, df_taxonomy=df_tax, pnns_n=2, nb_groups=3), axis=1)

Wall time: 6min 6s
Wall time: 6min 35s


In [13]:
def find_pnns_groups_1(df):
    """ Given a pnns group 2, find the corresponding pnns group 1."""
    data = df.copy()
    vals_to_find = list(data.pnns_groups_1.unique())
    vals_to_find.remove('unknown')
    for val in vals_to_find:
        group_2_vals = list(data['pnns_groups_2'].loc[data['pnns_groups_1'] == val].unique())
        data['pnns_groups_1'].loc[(data['pnns_groups_1'] == 'unknown') & 
        (data['pnns_groups_2'].isin(group_2_vals))] = val
    return data

In [14]:
pnns = [f'pnns_groups_{i}'for i in range (1,5)]
pnns

['pnns_groups_1', 'pnns_groups_2', 'pnns_groups_3', 'pnns_groups_4']

In [15]:
df.dropna(subset=pnns, inplace=True)

In [16]:
df_pnns = find_pnns_groups_1(df)

In [17]:
df_pnns.drop(index=690514, inplace=True) #was an error

In [18]:
#Clean pnns groups 2
df_pnns['pnns_groups_2'].loc[df_pnns['pnns_groups_2'] == 'plant-based milk substitutes'] = 'plant based milk substitutes'
df_pnns['pnns_groups_2'].loc[df_pnns['pnns_groups_2'] == 'one-dish meals'] = 'one dish meals'

In [19]:
df_pnns.shape

(674517, 186)

### Create a list with cols to keep for training and export

In [24]:
pnns.append('code')
pnns.append('url')
pnns.append('product_name')
pnns

['pnns_groups_1',
 'pnns_groups_2',
 'pnns_groups_3',
 'pnns_groups_4',
 'code',
 'url',
 'product_name']

In [25]:
data_export = df_pnns[pnns]
data_export.shape

(674517, 7)

In [26]:
data_export

,pnns_groups_1,pnns_groups_2,pnns_groups_3,pnns_groups_4,code,url,product_name
3,fat and sauces,dressings and sauces,dressings and sauces,dressings and sauces,0000000000100,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin
14,composite foods,one dish meals,legumes,legumes,0000000000949,http://world-en.openfoodfacts.org/product/0000...,Salade de carottes râpées
22,sugary snacks,biscuits and cakes,pizza pies and quiche,pizza pies and quiche,0000000001281,http://world-en.openfoodfacts.org/product/0000...,Tarte noix de coco
32,fruits and vegetables,fruits,fruits,dairy desserts,0000000001885,http://world-en.openfoodfacts.org/product/0000...,Compote de poire
34,fish meat eggs,meat,meat,meat,0000000002103,http://world-en.openfoodfacts.org/product/0000...,Aiguillettes de poulet
...,...,...,...,...,...,...,...
1625055,fish meat eggs,meat,meat,meat,9999992756068,http://world-en.openfoodfacts.org/product/9999...,Steak haché
1625058,sugary snacks,sweets,sweets,sweets,9999999004360,http://world-en.openfoodfacts.org/product/9999...,Minis beignets
1625060,sugary snacks,biscuits and cakes,biscuits and cakes,biscuits and cakes,9999999175305,http://world-en.openfoodfacts.org/product/9999...,Erdbeerkuchen 1019g tiefgefroren
1625061,fish meat eggs,meat,meat,meat,99999995,http://world-en.openfoodfacts.org/product/9999...,Steak haché pur boeuf


In [27]:
data_export.isnull().sum()

pnns_groups_1       0
pnns_groups_2       0
pnns_groups_3       0
pnns_groups_4       0
code                0
url                 0
product_name     5430
dtype: int64

## Export data to csv

In [29]:
data_export.to_csv (r'C:\Users\Antoine\Coding Bootcamp\Open Food Facts\df_multi_pnns_v2.csv', index = False, header=True)